# 要約 
このJupyterノートブックは、Kaggleの「LLM 20 Questions」コンペティションに参加するための言語モデルの構築を目的としています。このコンペティションでは、参加者が「20の質問」ゲームをプレイし、推測者と回答者の役割を果たす2つの言語モデル（LLM）を使用して、秘密の単語をできるだけ少ない質問で特定しようとします。

### 取り組んでいる問題
ノートブックでは、質問と回答のプロセスを模倣し、秘密の単語を推測するために戦略的な質問を生成するための手法を探究しています。具体的には、モデルは「はい」または「いいえ」でしか答えられない質問を通じて、情報を絞り込んでいくことを目指します。

### 使用されている手法やライブラリ
1. **データ処理**:
   - **Pandas**: データ操作のためのライブラリ。このノートブックではデータフレームの作成や操作に使用されています。
   - **NumPy**: 数値計算のためのライブラリ。

2. **自然言語処理**:
   - **KerasとKeras NLP**: 深層学習のためのフレームワークとしてKerasを使用し、特に自然言語処理用の拡張機能であるKeras NLPを利用しています。
   - **GemmaCausalLMモデル**: Keras NLPから事前に構成されたGemmaモデルを使用し、指定されたプロンプトに基づいてテキスト生成を行います。

3. **エージェントの設計**:
   - `Agent`クラスが定義されており、質問を生成し、回答を受け取って記録する機能を持っています。`ask_question`メソッドは質問をモデルに送り、`receive_answer`メソッドは受け取った回答を管理するための基礎を提供します。

### まとめ
このノートブックは、Kaggleの20の質問ゲームに挑むために、様々なライブラリと手法を使用して言語モデルを構築し、エージェントの設計や情報処理の実装に取り組んでいます。最終的な目標は、できるだけ少ない質問で秘密の単語を特定する能力を持ったエージェントを作成することです。

---


# 用語概説 
初心者がつまずきそうな専門用語の解説を以下に示します。これらの用語は、ノートブックの内容に特有であり、実務経験がないと馴染みが薄い可能性があります。

1. **LLM (Large Language Model)**:
   - 大規模な言語モデルの略で、膨大なデータセットから学習し、人間のような文を生成したり、質疑応答を行ったりできる機械学習モデルのこと。例えば、GPTやBERTがこれに該当します。

2. **演繹的推論**:
   - 具体的な事例から一般的な結論を導き出す推論方法。20の質問ゲームでは、最初の質問を一般的に設定し、それに対する回答から次の質問を絞り込む形式が演繹的推論の一例です。

3. **Keras**:
   - 高レベルなニューラルネットワークライブラリ。TensorFlowやTheanoをバックエンドとして利用し、モデルの構築や訓練を簡単に行えるように設計されています。

4. **Keras_NLP**:
   - Kerasを拡張した自然言語処理用のライブラリ。言語モデル、テキスト前処理、トークナイザーなど、自然言語処理に特化した機能を提供します。

5. **GemmaCausalLM**:
   - Keras_NLPで利用可能な特定の因果関係に基づく言語モデル。入力された情報に基づいて次に続くテキストを生成することができ、特に質問応答や文生成に使用されます。

6. **プロンプト**:
   - モデルに対する入力のこと。特に言語モデルにおいて、ユーザーがモデルに期待する応答を生成するための文やフレーズを指します。

7. **トークン**:
   - テキストの最小単位で、単語や部分的な単語を指すことが多い。言語モデルは、入力されたテキストをトークンに分割して処理します。

8. **知識の蓄積 (knowledge)**:
   - 質問や回答を記録するための情報の一覧。エージェントはゲームが進行する中でこの知識を利用して次の質問を戦略的に決定します。

9. **エージェント (Agent)**:
   - ゲーム内で特定の役割を果たすシステムやプログラムを指します。このノートブックでは、質問者の役割を果たすエージェントが定義されています。

10. **環境変数 (Environment Variables)**:
    - システムのオペレーティングシステムやソフトウェアの動作に影響を与える設定情報。ここではKerasのバックエンドを設定するために使用されています。

これらの用語の理解が、ノートブック内のコードやコンペティションの文脈を把握する上で役に立つでしょう。

---


## LLM 20 Questions

# 説明

それは人、場所、それとも物ですか？それはパン箱より小さいですか？それは70Bパラメータモデルより小さいですか？

「20の質問」は古くからある推論ゲームであり、20問以内で秘密の単語を推測しようとします。このゲームでは、はい・いいえで答えられる質問のみを使用します。プレイヤーは、質問を一般的なものから具体的なものへと絞り込みながら、その単語を推測することを試みます。できるだけ少ない質問で単語を当てることを目指します。

各チームは、質問をし推測を行う役割の「推測者LLM」と、「はい」または「いいえ」で応答する役割の「回答者LLM」の2つで構成されます。戦略的な質問と回答を通じて、推測者ができる限り少ないラウンドで秘密の単語を正確に特定することが目標です。

このコンペティションでは、演繹的推論、的を絞った質問による効率的な情報収集、ペアエージェント間の連携といったLLMの重要なスキルが評価されます。また、限られた推測回数で創造性と戦略を駆使する必要がある制約のある環境を提供します。成功すれば、LLMは単に質問に答えるだけでなく、洞察に富んだ質問をし、論理的な推論を行い、可能性を迅速に絞り込む能力を示すことになります。

In [ ]:
import pandas as pd  # pandasライブラリをインポートしてデータ操作を行う
import numpy as np  # numpyライブラリをインポートして数値計算を行う
from llm_20_questions.keywords import KEYWORDS_JSON  # 独自モジュールからKEYWORDS_JSONをインポート
import keras  # Kerasライブラリをインポートして深層学習モデルを構築する
import keras_nlp  # Kerasの自然言語処理関連機能を提供するライブラリをインポート

# pandasはデータ解析のためのライブラリで、特にデータフレームの作成・操作が得意です。
# numpyは多次元配列を扱うためのライブラリで、高速な数値計算が可能です。
# llm_20_questions.keywordsからKEYWORDS_JSONをインポートすることで、特定のキーワードデータにアクセスできます。
# kerasは深層学習のためのフレームワークで、ニューラルネットワークの構築・訓練ができる。
# keras_nlpは、自然言語処理に特化したKerasの拡張機能です。

In [ ]:
keywords = eval(KEYWORDS_JSON)  # KEYWORDS_JSONを評価してPythonのオブジェクトに変換し、keywords変数に格納する

# eval関数を使用することで、KEYWORDS_JSONに格納されているデータを実際のデータ構造に変換します。
# これにより、JSONフォーマットのキーワードデータをPythonの辞書形式として扱えるようになります。

In [ ]:
import os  # osライブラリをインポートしてオペレーティングシステムとの対話を行う

os.environ["KERAS_BACKEND"] = "tensorflow"  # 環境変数KERAS_BACKENDを'TensorFlow'に設定する

# ここでは、KerasのバックエンドとしてTensorFlowを使用するように指定しています。
# これにより、Kerasで構築したモデルがTensorFlowを利用して動作するようになります。
# 複数のバックエンドが利用可能な場合、環境変数を設定することで選択を行います。

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")  # GemmaCausalLMモデルをプリセットから読み込む

# keras_nlp.models.GemmaCausalLMは、特定の自然言語処理タスクに特化したモデルです。
# from_presetメソッドを使用することで、事前に定義された設定（ここでは"gemma_2b_en"）を持つモデルインスタンスを生成します。
# このモデルは、与えられた情報を基に文を生成したり、テキストの解析を行うために使用されます。

In [ ]:
gemma_lm.summary()  # GemmaCausalLMモデルの概要を出力する

# summaryメソッドを使用することで、モデルの構造や層の詳細、パラメータ数などの情報が表示されます。
# これにより、モデルのアーキテクチャを理解し、適切に使用するための手助けとなります。
# モデルの性能や学習状況を把握するために重要なステップです。

In [ ]:
gemma_lm.generate(KEYWORDS_JSON, max_length=20)  # KEYWORDS_JSONを入力として、最大20トークンの生成を行う

# generateメソッドは、与えられた入力データ（ここではKEYWORDS_JSON）に基づいて新しいテキストを生成します。
# max_length引数は生成されるテキストの最大長を指定し、ここでは20トークンまでの長さに制限しています。
# この方法を使って、モデルがキーワードに関連する文を生成することが期待されます。

In [ ]:
promt = """
    Hello. Now we play game 20 Questions. I have words and you must guess that. 
    I can answer you only "Yes" or "No". Please, ask me?
    """  # 「こんにちは。今から20の質問ゲームをプレイします。私は言葉を持っており、あなたはそれを推測しなければなりません。
          # 私は「はい」または「いいえ」だけで答えることができます。どうぞ、私に質問してください？」
          # というプロンプトを定義している。
          
# このプロンプトは、ゲームの開始を告げ、質問を促す内容で構成されています。
# プレイヤーが質問を開始できるよう、ルールを簡潔に説明しています。

In [ ]:
gemma_lm.generate(promt, max_length=15)  # プロンプトを入力として、最大15トークンの生成を行う

# generateメソッドを使用して、定義したプロンプト（ゲームの開始を告げる内容）に基づいて新しいテキストを生成します。
# max_length引数は生成されるテキストの最大長を指定し、ここでは15トークンまでの長さに制限しています。
# この処理は、ゲームの進行に必要な初回の応答を生成するために使用されることが期待されます。

In [ ]:
class Agent:
    def __init__(self):
        self.gemma = gemma_lm  # GemmaCausalLMモデルのインスタンスを保持
        self.knowledge = []  # 質問と回答を記録するためのリストを初期化

    def ask_question(self, question):
        self.knowledge.append(question)  # 提出された質問をknowledgeリストに追加
        response = self.gemma.generate(question)  # Gemmaモデルを使用して質問に対する応答を生成
        return response  # 生成された応答を返す

    def receive_answer(self, answer):
        self.knowledge.append(answer)  # 提出された回答をknowledgeリストに追加
        # 必要に応じて回答を処理する
        pass  # 将来的にこのメソッドを拡張するためのプレースホルダー

# Agentクラスは、ゲームにおける質問者を定義しており、質問の記録と応答生成を行います。
# ask_questionメソッドは質問を送信し、その応答を受け取る機能を提供します。
# receive_answerメソッドは、受け取った回答を処理するための基礎を提供します。